In [1]:
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk

# Load the CSV file into a DataFrame
data = pd.read_csv('data/mental_feed.csv')
import pandas as pd

# Load the DataFrame
data = pd.read_csv('data/mental_feed.csv')

In [ ]:
# Define function to tokenize sentences and words
def tokenize_sentence(text):
    sentence_tokens = nltk.sent_tokenize(text)
    return sentence_tokens
def tokenize_word(text):
    word_tokens = nltk.word_tokenize(text)
    return word_tokens

# Apply tokenization to each question and answer
data['Question_Sentence_Tokens'] = data['question'].apply(tokenize_sentence)
data['Question_Word_Tokens'] = data['question'].apply(tokenize_word)
data['Answer_Sentence_Tokens'] = data['answer'].apply(tokenize_sentence)
data['Answer_Word_Tokens'] = data['answer'].apply(tokenize_word)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['question'])

In [ ]:
import re

def hybrid_chatbot(user_input):
    patterns = {
        "greet": r"hello|hi|hey",
        "thanks": r"thank you|thanks"
    }
    responses = {
        "greet": "Hello! How can I help you today?",
        "thanks": "You're welcome!"
    }
    
    # Check for pattern matches first
    for intent, pattern in patterns.items():
        if re.search(pattern, user_input, re.IGNORECASE):
            return responses[intent]
    
    # Tokenize user input
    user_input_sentence_tokens = tokenize_sentence(user_input) 
    user_input_word_tokens = tokenize_word(user_input)
    
    # Convert tokenized input to string for vectorization
    user_input_str = ' '.join(user_input_word_tokens)
    
    # Transform the user input using the vectorizer
    user_input_vector = vectorizer.transform([user_input_str])
    
    # Calculate similarity between user input and all questions
    similarities = cosine_similarity(user_input_vector, X).flatten()
    
    # Find the index of the most similar question
    most_similar_index = similarities.argmax()
    
    # Fetch the answer corresponding to the most similar question
    response = data.iloc[most_similar_index]['answer']
    return response
# print("Chatbot: Hello there! Tell me how you feel and I will try to help you with it")
# Testing the chatbot
while True:
    user_input = input("=====>You: ")
    if user_input.lower() == r"bye|goodbye|end|quit|exit|terminate|kill yourself|i hate you|you are dumb":
        print("Chatbot: Goodbye, take care.")
        break
    response = hybrid_chatbot(user_input)
    print("Chatbot:", response)
